In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tensorflow.keras import layers, Sequential, mixed_precision
from tensorflow import keras


In [26]:
mixed_precision.set_global_policy('mixed_float16')


In [2]:

def read_csv(file_path):
    '''
    Read CSV file and return data as numpy array
    '''
    df = pd.read_csv(file_path)
    if "class" in list(df.columns.values):
        df = df.drop("class", axis=1)
    data = np.array(df,dtype=object)
    data = data.astype(float)
    return data

In [4]:
data = read_csv("init_demo.csv")
data.shape

(1, 300)

In [5]:
def find_max_for_one_file(data, var):
    current_max = 0
    vars = {"x": 0, "y": 1, "z": 2}
    for i in range(data.shape[0]):
        frame = data[i]
        frame = frame.reshape((-1, 4))
        for j in range(frame.shape[0]):
            if frame[j][vars[var]] > current_max:
                current_max = frame[j][vars[var]]
    
    return current_max
find_max_for_one_file(data, "x")

0.8602175712585449

In [6]:
def initialize_matrix_for_frame(data, idx, x_dim, y_dim):
   
    # create a matrix of all zeros
    matrix = np.zeros((x_dim//10,y_dim//10))
    # fill the matrix with random values
    frame = data[idx]
    frame = frame.reshape((-1, 4))
    for j in range(len(frame)):
        matrix[int(frame[j][0] * 1000/10)][int(frame[j][1] * 1000/10)] = j
    return matrix

In [132]:
matrix = initialize_matrix_for_frame(data, 0)

TypeError: initialize_matrix_for_frame() missing 2 required positional arguments: 'x_dim' and 'y_dim'

In [7]:
# plot the matrix
coords = np.nonzero(matrix)

# Plot the non-zero elements
plt.scatter(coords[0], coords[1])

# Set the x and y limits
plt.xlim(0, matrix.shape[0])
plt.ylim(0, matrix.shape[1])

# Set the labels for the axes
plt.xlabel('Row number')
plt.ylabel('Column number')

# Show the plot
plt.show()

NameError: name 'matrix' is not defined

In [8]:
def generate_tensor(data, x_dim=1500, y_dim=3200, sliding_window_size=10):
    # check for max value in x and y, which will then be used for the shape of the matrix

    for i in range(len(data) - sliding_window_size + 1):
        # print(i, len(data) - sliding_window_size + 1)
        tensor = np.zeros((sliding_window_size, x_dim//10,y_dim//10))
        for j in range(i, i + 10):
            new_matrix = initialize_matrix_for_frame(data, j, x_dim, y_dim)
            tensor[j-i] = new_matrix
        yield tensor
    

In [10]:
test_arr = np.empty((0,10,150,320))
tensor_gen = generate_tensor(data, 1500, 3200, sliding_window_size=10)
arr_to_add = np.expand_dims(next(tensor_gen), axis=0)
test_arr = np.concatenate((test_arr,  arr_to_add), axis=0)
test_arr.shape

StopIteration: 

In [18]:

# path = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/data_augmentation/data_four_labels_augmentation"
path = "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/data_augmentation/data_four_labels_augmentation"
def find_max_dim__for_entire_folder(path, sliding_window_size = 10):
    # The first thing to do is find the maximum x and y values for our matrices
    max_x_dim = 0
    max_y_dim = 0
    for label in os.listdir(path):
        if label != ".DS_Store":
            i = 1
            j = 0
            for file in os.listdir(os.path.join(path, label)):
                # pick 25% of the data
                if i%4 == 0:
                    data = read_csv(os.path.join(path, label, file))
                    max_x = find_max_for_one_file(data, "x") * 1000
                    max_y = find_max_for_one_file(data, "y") * 1000
                    x_dim = round(math.ceil(max_x/100)*100, -1)
                    y_dim = round(math.ceil(max_y/100)*100, -1)
                    if max_x_dim < x_dim:
                        max_x_dim = x_dim
                    if max_y_dim < y_dim:
                        max_y_dim = y_dim
                i += 1
        print(label, max_x_dim, max_y_dim)
    return max_x_dim, max_y_dim
# find_max_dim__for_entire_folder(path, sliding_window_size = 10)
                            

In [32]:
 # 80/20 split
x_dim = 1500
y_dim = 3200
sliding_window_size = 10

def process_entire_folder(path, sliding_window_size = 10, x_dim=1500, y_dim=3200):
    # The first thing to do is find the maximum x and y values for our matrices
    global X_train
    global y_train
    global X_val
    global y_val
    X_train = np.empty((0,10,x_dim//10,y_dim//10))
    y_train = []
    X_val = np.empty((0,10,x_dim//10,y_dim//10))
    y_val = []
    for label in os.listdir(path):
        if label != ".DS_Store":
            i = 1
            j = 0
            for file in os.listdir(os.path.join(path, label)):
                # pick 25% of the data
                if i%100 == 0:
                    j += 1
                    print(j, i, label, file)
                    file_path = os.path.join(path, label, file)
                    data = read_csv(file_path)
                    if np.isnan(data).any():
                        continue
                    tensor_generator = generate_tensor(data, x_dim, y_dim, sliding_window_size)
                    for tensor in tensor_generator:
                        arr_to_add = np.expand_dims(tensor, axis=0)
                        if j%5 == 0:
                            # add to validation set
                            X_val = np.concatenate((X_val,  arr_to_add), axis=0)
                            y_val.append(label)
                        else:
                            X_train = np.concatenate((X_train,  arr_to_add), axis=0)
                            y_train.append(label)
                i += 1
process_entire_folder(path, sliding_window_size, x_dim, y_dim)

1 100 coffee ax2UGtA8h3E2137_reflection.csv
2 200 coffee blZ0jXHGbYI0_reflection_rotation_15.csv_projective_geo_13_True.csv
3 300 coffee c0tloO5tJUY7125_reflection_rotation_15_scale_0.2.csv_projective_geo_7_False.csv
4 300 coffee c0tloO5tJUY7125_reflection_rotation_15_scale_0.2_translate_0.5_left.csv
5 400 coffee c0tloO5tJUY7209_reflection_rotation_15_translate_0.5_left.csv_projective_geo_13_False.csv
6 500 coffee CSj7IScvZnE5770_reflection_rotation_15_translation_0.5.csv_projective_geo_8_False.csv
7 600 coffee CxTSVyM-ij013680_reflection_rotation_15_translation_0.5_scale_0.2_translate_0.5_left.csv
8 700 coffee CxTSVyM-ij013790_reflection_rotation_15_translation_0.5_translate_0.5_left.csv_projective_geo_13_True.csv
9 800 coffee eeHS78JyN706789_reflection_scale_0.2.csv_projective_geo_10_True.csv
10 900 coffee eeHS78JyN706909_reflection_translate_0.5_left.csv
11 1000 coffee FEZ2sZmYRbI0_reflection_translation_0.5.csv_projective_geo_11_False.csv
12 1100 coffee fKjsdtMU3fc8470_reflection_t

KeyboardInterrupt: 

In [20]:
X_val.shape

(0, 10, 150, 320)

In [24]:
y_train

[]

In [17]:
X_train.shape

(0, 10, 150, 320)

In [45]:
x_dim = 1500
y_dim = 3200
sliding_window_size = 10

def process_entire_folder(path, sliding_window_size = 10, x_dim=1500, y_dim=3200):
    # The first thing to do is find the maximum x and y values for our matrices
    global X_train
    global y_train
    global X_val
    global y_val
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    for label in os.listdir(path):
        if label != ".DS_Store":
            i = 1
            j = 0
            for file in os.listdir(os.path.join(path, label)):
                # pick 25% of the data
                if i%1000 == 0:
                    j += 1
                    print(j, i, label, file)
                    tensor_generator = generate_tensor(data, x_dim, y_dim, sliding_window_size)
                    for tensor in tensor_generator:
                        
                        if j%5 == 0:
                            # add to validation set
                            X_val.append(tensor)
                            y_val.append(label)
                        else:
                            X_train.append(tensor)
                            y_train.append(label)
                i += 1
process_entire_folder(path, sliding_window_size, x_dim, y_dim)

1 1000 dog g59_3vr_NT01632_translate_0.5_left.csv_projective_geo_7_True.csv
2 2000 dog rIlQQTN2b586887_reflection_rotation_15_translation_0.5_scale_0.2.csv
3 3000 dog sUC-XDC5TvA2228_reflection_rotation_15.csv_projective_geo_9_True.csv
1 1000 milk jDCw7stJaM43664_reflection_translation_0.5.csv_projective_geo_14_True.csv
2 2000 milk wjbWyQLPqwQ0_translation_0.5.csv_projective_geo_6_True.csv
3 3000 milk MIPvaUKFA4c750_reflection_translation_0.5_translate_0.5_left.csv
4 4000 milk 6UrcyZ-QeiU550_scale_0.2.csv_projective_geo_7_False.csv
1 1000 coffee eeHS78JyN706909_rotation_15_translate_0.5_left.csv
2 2000 coffee FUHOIo_8o9w4459_translation_0.5.csv
1 1000 door nHnqAu_1K-c12221_reflection_translation_0.5.csv_projective_geo_12_True.csv
2 2000 door K4lo0NZDlH86823_scale_0.2_translate_0.5_left.csv


In [20]:
def reformat_label(label_arr):    
#should be between 0 and 3
    dicti={"coffee": 0, "dog": 1, "door": 2, "milk": 3}
    # for element in df_labels:
    #     element = dicti[element]
    for i in range(len(label_arr)):
        label_arr[i] = dicti[label_arr[i]]
    return label_arr
y_train = reformat_label(y_train)
y_val = reformat_label(y_val)

In [21]:
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train, y_train, random_state=0)
X_val,Y_val = shuffle(X_val, y_val, random_state=0)
X_train, Y_train


(array([[[[74.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[74.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[74.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         ...,
 
         [[74.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  

In [22]:
Y_train = np.array(Y_train).reshape((-1, 1))
Y_val = np.array(Y_val).reshape((-1, 1))
print(Y_train.shape, Y_val.shape)

(770, 1) (175, 1)


In [13]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/mediapipe/Sliding_window_dense-Layers/AUGMENTED-CNN.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [28]:
model = Sequential()
model.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(10, 150, 320, 1)))
model.add(layers.MaxPooling3D((5, 75, 160), padding="same"))
# model.add(layers.Conv3D(64,(2,2,2),activation='relu'))
# model.add(layers.MaxPooling3D((2, 38, 80), padding="same"))
model.add(layers.Dropout(0.6))
model.add(layers.Flatten())
model.add(layers.Dense(256,'relu'))
model.add(layers.Dense(128,'relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(128,'relu'))
# model.add(layers.Dense(64,'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4,'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'],
             )
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 8, 148, 318, 32)   896       
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 2, 2, 2, 32)      0         
 3D)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 2, 2, 2, 32)       0         
                                                                 
 flatten_5 (Flatten)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 256)               65792     
                                                                 
 dense_12 (Dense)            (None, 128)               32896     
                                                      

In [29]:
# Fit data to model
history = model.fit(X_train, Y_train,
            batch_size=32,
            epochs=40,
            verbose=1,
            validation_data=(X_val,Y_val), callbacks=[checkpoint])

Epoch 1/40


InvalidArgumentError: No OpKernel was registered to support Op 'MaxPool3D' used by {{node sequential_5/max_pooling3d_4/MaxPool3D}} with these attrs: [ksize=[1, 5, 75, 160, 1], padding="SAME", T=DT_HALF, strides=[1, 5, 75, 160, 1], data_format="NDHWC"]
Registered devices: [CPU, GPU]
Registered kernels:
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_BFLOAT16, DT_HALF]
  device='CPU'; T in [DT_FLOAT]

	 [[sequential_5/max_pooling3d_4/MaxPool3D]] [Op:__inference_train_function_3726]